# Example about how to add a dataset catalog to OpenVisus

Using the idea of `visus.config` but centrally managed in a catalog file.
To maintain backward compatibility, the catalog is stored in XML format, but it can be migrated to JSON and/or YAML in the future.


## Import the necessary libraries



In [2]:
import os,sys,logging,time
import numpy as np

# needed only in debug mode
debug_mode=True
if debug_mode:
  sys.path.append("c:/projects/openvisus/build/RelWithDebInfo")
  sys.path.append("c:/projects/openvisuspy/src")

import OpenVisus as ov
import openvisuspy
print(time.time(),"OpenVisus imported")

1751875449.7328525 OpenVisus imported


## Catalog source code, to be integrated into openvisuspy

In [ ]:
import os,requests

# ///////////////////////////////////////////////////////////////////////////
class VisusCatalog:
   
  # constructor
  def __init__(self, stree):
     self.stree=stree
     
  # loadDataset
  def loadDataset(self, name:str):
     
    N=self.stree.getNumberOfChilds()
    for I in range(N):
        child = self.stree.getChild(I)
        if child.name=="dataset" and child.getAttribute("name") == name:
            return ov.LoadDatasetEx(child)

    raise ValueError(f"loadDataset '{name}' not found in the catalog.")


# ///////////////////////////////////////////////////////////////////////////
def LoadCatalog(url:str=None)-> VisusCatalog:

  """
  url: str or None
  - None: use the default catalog
  - str: URL to the catalog, can be a local file path (i.e. ~/visus.config) or a remote URL (`http://nationalsciencedatafabric.org/catalog.config`).
  - str: body of the catalog in XML format (to be extended to JSON and YAML too?).
  """

  # no URL provided, use the default catalog
  if url is None:

    # TODO: donload from a default remote URL (what is a good default URL???)
    # ....
    if True:
      pass

    # failed, swith to default catalog that should be inside the OpenVisus package
    default_catalog_filename=os.path.join(os.path.dirname(ov.__file__),"catalog.config")

    # backward compatible: create if does not exist
    if not os.path.exists(default_catalog_filename):
      with open(default_catalog_filename, 'w') as f:
         print("Writing default catalog to", default_catalog_filename)
         f.write("""<datasets>
  <dataset name='2kbit1' url='http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1' >
    <access type='network' chmod='r' compression='zip'  />
  </dataset>
</datasets>""")
    
    return LoadCatalog(default_catalog_filename)

  # url is a local file path
  local_path=os.path.expanduser(url)
  if os.path.exists(local_path):
    print("Loading catalog from", local_path)
    with open(local_path, 'r') as f:
        body=f.read()

  # url is a remote URL
  elif url.startswith('http://') or url.startswith('https://'):
    print("Loading catalog from remote URL", url)
    response = requests.get(url)
    response.raise_for_status()
    body=response.text
    
  # url is a body of the catalog in XML format
  elif url.strip().startswith("<") and url.strip().endswith(">"):
    print("Assuming url is a body of the catalog in XML format")
    body=url

  else:
    raise ValueError("Invalid url. Please provide None | valid url | XML document.")
  
  # parse XML body
  print("Parsing catalog body")
  body = body.strip()
  print(body)
  try:
    stree=ov.StringTree.fromString(body)
    print("Parsed StringTree:", stree.valid())
    if stree.valid():
      return VisusCatalog(stree)
  except Exception as e:
    print("Exception while parsing StringTree:", e)
    pass

  raise ValueError("Error parsing StringTree from the provided body, trying to parse as XML")


## Example of usage of default catalog

In [54]:
catalog=LoadCatalog()
db=catalog.loadDataset("2kbit1")
body=db.getDatasetBody().toString()
print(body)

Loading catalog from c:\projects/openvisus/build/RelWithDebInfo\OpenVisus\catalog.config
Parsing catalog body
<datasets>
  <dataset name='2kbit1' url='http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1' >
    <access type='network' chmod='r' compression='zip'  />
  </dataset>
</datasets>
Parsed StringTree: True
<dataset name="2kbit1" url="http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1" typename="IdxDataset">
	<access type="network" chmod="r" compression="zip" />
	<idxfile>
		<version value="6" />
		<bitmask value="V012012012012012012012012012012012" />
		<box value="0 2048 0 2048 0 2048" />
		<bitsperblock value="16" />
		<blocksperfile value="256" />
		<block_interleaving value="0" />
		<filename_template value="./visus/%02x/%04x.bin" />
		<missing_blocks value="False" />
		<arco value="0" />
		<time_template value="" />
		<axis value="" />
		<field name="DATA" description="" index="" default_compression="zip" default_layout="hzorder" default_value="0" filter="" dtype="uint8

## Load the catalog from a URL or local file

In [56]:
catalog=LoadCatalog("../catalogs/default.config")
db=catalog.loadDataset("2kbit1")
body=db.getDatasetBody().toString()
print(body)

Loading catalog from ../catalogs/default.config
Parsing catalog body
<datasets>
      <dataset name='2kbit1' url='http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1' >
          <access type='network' chmod='r' compression='zip'  />
      </dataset>
  </datasets>
Parsed StringTree: True
<dataset name="2kbit1" url="http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1" typename="IdxDataset">
	<access type="network" chmod="r" compression="zip" />
	<idxfile>
		<version value="6" />
		<bitmask value="V012012012012012012012012012012012" />
		<box value="0 2048 0 2048 0 2048" />
		<bitsperblock value="16" />
		<blocksperfile value="256" />
		<block_interleaving value="0" />
		<filename_template value="./visus/%02x/%04x.bin" />
		<missing_blocks value="False" />
		<arco value="0" />
		<time_template value="" />
		<axis value="" />
		<field name="DATA" description="" index="" default_compression="zip" default_layout="hzorder" default_value="0" filter="" dtype="uint8" />
		<timestep when="0"

## Example of usage using a remote catalog file

In [53]:
catalog=LoadCatalog("https://raw.githubusercontent.com/sci-visus/openvisuspy/refs/heads/main/catalogs/default.config")
db=catalog.loadDataset("2kbit1")
body=db.getDatasetBody().toString()
print(body)

Loading catalog from remote URL https://raw.githubusercontent.com/sci-visus/openvisuspy/refs/heads/main/catalogs/default.config
Parsing catalog body
<datasets>
      <dataset name='2kbit1' url='http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1' >
          <access type='network' chmod='r' compression='zip'  />
      </dataset>
  </datasets>
Parsed StringTree: True
<dataset name="2kbit1" url="http://atlantis.sci.utah.edu/mod_visus?dataset=2kbit1" typename="IdxDataset">
	<access type="network" chmod="r" compression="zip" />
	<idxfile>
		<version value="6" />
		<bitmask value="V012012012012012012012012012012012" />
		<box value="0 2048 0 2048 0 2048" />
		<bitsperblock value="16" />
		<blocksperfile value="256" />
		<block_interleaving value="0" />
		<filename_template value="./visus/%02x/%04x.bin" />
		<missing_blocks value="False" />
		<arco value="0" />
		<time_template value="" />
		<axis value="" />
		<field name="DATA" description="" index="" default_compression="zip" default_la